# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Bañados  ->  E. Bañados  |  ['E. Bañados']
F. Davies  ->  F. Davies  |  ['F. Davies']
S. Li  ->  S. Li  |  ['S. Li']
J. He  ->  J. He  |  ['J. He']
Arxiv has 61 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2311.08451


extracting tarball to tmp_2311.08451...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3548: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3548: LatexWarning: Found documentclass in tmp_2311.08451/axis_highz_jets.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

 done.


Found 53 bibliographic references in tmp_2311.08470/feedback_and_igm.bbl.
Retrieving document from  https://arxiv.org/e-print/2311.08651


extracting tarball to tmp_2311.08651...

 done.


Found 94 bibliographic references in tmp_2311.08651/ms.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.08470-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.08470) | **Searching for the Imprints of AGN Feedback on the Lyman Alpha Forest  Around Luminous Red Galaxies**  |
|| V. Khaire, et al. -- incl., <mark>F. Davies</mark> |
|*Appeared on*| *2023-11-16*|
|*Comments*| *21 pages (including 4 page appendix), Submitted to MNRAS*|
|**Abstract**| We explore the potential of using the low-redshift Lyman-$\alpha$ (Ly$\alpha$) forest surrounding luminous red galaxies (LRGs) as a tool to constrain active galactic nuclei (AGN) feedback models. Our analysis is based on snapshots from the Illustris and IllustrisTNG simulations at a redshift of $z=0.1$. These simulations offer an ideal platform for studying the influence of AGN feedback on the gas surrounding galaxies, as they share the same initial conditions and underlying code but incorporate different feedback prescriptions. Both simulations show significant impacts of feedback on the temperature and density of the gas around massive halos. Following our previous work, we adjusted the UV background in both simulations to align with the observed number density of Ly$\alpha$ lines ($\rm dN/dz$) in the intergalactic medium and study the Ly$\alpha$ forest around massive halos hosting LRGs, at impact parameters ($r_{\perp}$) ranging from 0.1 to 100 pMpc. Our findings reveal that $\rm dN/dz$, as a function of $r_{\perp}$, is approximately 1.5 to 2 times higher in IllustrisTNG compared to Illustris up to $r_{\perp}$ of $\sim 10$ pMpc. To further assess whether existing data can effectively discern these differences, we search for archival data containing spectra of background quasars probing foreground LRGs. Through a feasibility analysis based on this data, we demonstrate that ${\rm dN/dz} (r_{\perp})$ measurements can distinguish between feedback models of IllustrisTNG and Illustris with a precision exceeding 12$\sigma$. This underscores the potential of ${\rm dN/dz} (r_{\perp})$ measurements around LRGs as a valuable benchmark observation for discriminating between different feedback models. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.08651-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.08651) | **The ALMA-QUARKS survey: -- I. Survey description and data reduction**  |
|| X. Liu, et al. -- incl., <mark>S. Li</mark>, <mark>J. He</mark> |
|*Appeared on*| *2023-11-16*|
|*Comments*| *9 figures, 4 tables, accepted by RAA*|
|**Abstract**| This paper presents an overview of the QUARKS survey, which stands for `Querying Underlying mechanisms of massive star formation with ALMA-Resolved gas Kinematics and Structures'. The QUARKS survey is observing 139 massive clumps covered by 156 pointings at ALMA Band 6 ($\lambda\sim$ 1.3 mm). In conjunction with data obtained from the ALMA-ATOMS survey at Band 3 ($\lambda\sim$ 3 mm), QUARKS aims to carry out an unbiased statistical investigation of massive star formation process within protoclusters down to a scale of 1000 au. This overview paper describes the observations and data reduction of the QUARKS survey, and gives a first look at an exemplar source, the mini-starburst Sgr B2(M). The wide-bandwidth (7.5 GHz) and high-angular-resolution (~0.3 arcsec) observations of the QUARKS survey allow to resolve much more compact cores than could be done by the ATOMS survey, and to detect previously unrevealed fainter filamentary structures. The spectral windows cover transitions of species including CO, SO, N$_2$D$^+$, SiO, H$_{30}\alpha$, H$_2$CO, CH$_3$CN and many other complex organic molecules, tracing gas components with different temperatures and spatial extents. QUARKS aims to deepen our understanding of several scientific topics of massive star formation, such as the mass transport within protoclusters by (hub-)filamentary structures, the existence of massive starless cores, the physical and chemical properties of dense cores within protoclusters, and the feedback from already formed high-mass young protostars. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.08451-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.08451) | **Uncovering the First AGN Jets with AXIS**  |
|| T. Connor, <mark>E. Bañados</mark>, N. Cappelluti, A. Foord |
|*Appeared on*| *2023-11-16*|
|*Comments*| *7 Pages, 2 Figures. This White Paper is part of a series commissioned for the AXIS Probe Concept Mission; additional AXIS White Papers can be found at the AXIS website (this http URL) with a mission overview here: arXiv:2311.00780*|
|**Abstract**| Jets powered by AGN in the early Universe ($z \gtrsim 6$) have the potential to not only define the trajectories of the first-forming massive galaxies but to enable the accelerated growth of their associated SMBHs. Under typical assumptions, jets could even rectify observed quasars with light seed formation scenarios; however, not only are constraints on the parameters of the first jets lacking, observations of these objects are scarce. Owing to the significant energy density of the CMB at these epochs capable of quenching radio emission, observations will require powerful, high angular resolution X-ray imaging to map and characterize these jets. As such, \textit{AXIS} will be necessary to understand early SMBH growth and feedback. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2311.08470.md
    + _build/html/tmp_2311.08470/./figures/kde_comparison_dist_0.66_1.10Mpc.png
    + _build/html/tmp_2311.08470/./figures/dndz_z01_12_145_with_data.png
    + _build/html/tmp_2311.08470/./figures/kde_comparison_dist_2.42_3.08Mpc.png
exported in  _build/html/2311.08651.md
    + _build/html/tmp_2311.08651/./sgrb2_cores_labeled.png
    + _build/html/tmp_2311.08651/./sgrb2_band6_conti_selfcalcompare.png
    + _build/html/tmp_2311.08651/./I08448_band6_conti_mergecompare.png
    + _build/html/tmp_2311.08651/./SgrB2_colorimages.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\HI}{H {\sc i}}$
$\newcommand{\HII}{\ion{H}{2}}$
$\newcommand{\HeI}{\ion{He}{1}}$
$\newcommand{\HeII}{\ion{He}{2}}$
$\newcommand{\HeIII}{\ion{He}{3}}$
$\newcommand{\SiIII}{\ion{Si}{3}}$
$\newcommand{\gHI}{\Gamma_{\ion{H}{1}}}$
$\newcommand{\nhi}{N_{\rm H I}}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\mathHI}{{\mbox{\scriptsize \HI}}}$
$\newcommand{\mathHII}{{\mbox{\scriptsize \HII}}}$
$\newcommand{\mathHeI}{{\mbox{\scriptsize \HeI}}}$
$\newcommand{\mathHeII}{{\mbox{\scriptsize \HeII}}}$
$\newcommand{\mathHeIII}{{\mbox{\scriptsize \HeIII}}}$
$\newcommand{\mathSiIII}{{\mbox{\scriptsize \SiIII}}}$
$\newcommand{\Msun}{M\textsubscript{\(\odot\)}}$</div>



<div id="title">

# $\huge$Searching for the Imprints of AGN Feedback on the Lyman AlphaForest Around Luminous Red Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2311.08470-b31b1b.svg)](https://arxiv.org/abs/2311.08470)<mark>Appeared on: 2023-11-16</mark> -  _21 pages (including 4 page appendix), Submitted to MNRAS_

</div>
<div id="authors">

V. Khaire, et al. -- incl., <mark>F. Davies</mark>

</div>
<div id="abstract">

**Abstract:** We explore the potential of using the low-redshift Lyman- $\alpha$ (Ly $\alpha$ ) forest surroundingluminous red galaxies (LRGs) as a tool to constrain active galactic nuclei (AGN)feedback models. Our analysis is based on snapshots from the Illustris and IllustrisTNGsimulations at a redshift of $z=0.1$ . These simulations offer an ideal platform for studying the influence of AGN feedbackon the gas surrounding galaxies, as they share the same initial conditions and underlying codebut incorporatedifferent feedback prescriptions.Both simulations show significant impacts of feedback on the temperature and density ofthe gas around massive halos. Following our previous work, we adjusted the UV background inboth simulations to align with the observed number density of Ly $\alpha$ lines ( $\rm dN/dz$ ) inthe intergalactic medium and  study the Ly $\alpha$ forest around massive halos hostingLRGs, at impact parameters ( $r_{\perp}$ ) ranging from 0.1 to 100 pMpc.Our findings reveal that $\rm dN/dz$ , as a function of $r_{\perp}$ , isapproximately 1.5 to 2 times higher in IllustrisTNG compared to Illustris up to $r_{\perp}$ of $\sim 10$ pMpc. To further assess whether existing data can effectivelydiscern these differences, we search for archival data containing spectra of background quasarsprobing foreground LRGs. Through a feasibility analysis based on this data,we demonstrate that ${\rm dN/dz} (r_{\perp})$ measurements can distinguish betweenfeedback models of IllustrisTNG and Illustris with a precision exceeding 12 $\sigma$ .This underscores the potential of ${\rm dN/dz} (r_{\perp})$ measurements around LRGsas a valuable benchmark observation for discriminating between different feedback models.

</div>

<div id="div_fig1">

<img src="tmp_2311.08470/./figures/kde_comparison_dist_0.66_1.10Mpc.png" alt="Fig6" width="100%"/>

**Figure 6. -** The KDE estimated 2D $b-$\nhi distribution for the halo sightlines
(within $\pm 500$ km $s^{-1}$) drawn from  the impact parameter bin
$0.6 <r{_\perp}< 1$ pMpc
of Illustris (left-hand panel)
and the IllustrisTNG (right-hand panel) halos for perfect (top)
and forward (bottom) models.
The distributions from perfect and forward models look similar expect
the normalization shown with colors.
In the bottom panel, we show $b-$\nhi distribution from perfect models
with contours. The forward models shift $b-$\nhi distribution  towards higher
$\nhi$ and lower $b$ values in both simulations.  (*fig.kde_small_impact*)

</div>
<div id="div_fig2">

<img src="tmp_2311.08470/./figures/dndz_z01_12_145_with_data.png" alt="Fig1" width="100%"/>

**Figure 1. -** The line density profile
${\rm dN/dz} (r_{\perp})$ for $10^{12} <N_{\rm HI} < 10 ^{14.5}  { \rm cm^{-2} }$ absorbers
obtained from
forward modeled mock spectra around LRG host
halos within $\pm 500 {\rm km s^{-1}}$ along line-of-sights.
IllustrisTNG (blue) gives a factor of $\sim 1.5-2$ higher
${\rm dN\slash dz}$ as compared to Illustris (red)
out to $r_\perp\lesssim$$5$$ {\rm pMpc}$($\sim 10   R_{\rm vir}$).
By construction, both simulations converge to the expected IGM ${\rm dN/dz}$(dotted lines) at large distances from halos.
The black points with error bars show the expected precision using
archival data assuming IllustrisTNG is the true model (see
section \ref{sec.data}
for more details). With this data, we can distinguish between
Illustris and IllustrisTNG
feedback models at 12$\sigma$ statistical significance. (*fig.dndz*)

</div>
<div id="div_fig3">

<img src="tmp_2311.08470/./figures/kde_comparison_dist_2.42_3.08Mpc.png" alt="Fig7" width="100%"/>

**Figure 7. -** 
The KDE estimated 2D $b-$\nhi distribution for the halo sightlines
(within $\pm 500$ km $s^{-1}$) drawn from  the impact parameter bin
$2.2 <r{_\perp}< 2.8$ pMpc
of Illustris (left-hand panel)
and the IllustrisTNG (right-hand panel) halos for perfect (top)
and forward (bottom) models.
The effect of feedback can be seen
in the difference in the distributions (in top panels) for perfect sightlines.
The hot gas in the Illustris
results in many absorption lines with low $\nhi$ high and $b$ values whereas
the  IllustrisTNG
shows both the low $\nhi$-high $b$ and high $\nhi$-low $b$ absorption lines.
These differences in the shape of $b-$\nhi get washed out in forward-modelled
sightlines as shown in the bottom panel where the black contours indicate
distribution from the perfect model shown on the top panel. However, the normalization of
distribution showing different numbers of lines,
indicated by the color bars is still
significantly different even in the distribution from forward-modeled sightlines.  (*fig.kde_intermediate_impact*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2311.08470"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$</div>



<div id="title">

# The ALMA-QUARKS survey:-- I. Survey description and data reduction

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2311.08651-b31b1b.svg)](https://arxiv.org/abs/2311.08651)<mark>Appeared on: 2023-11-16</mark> -  _9 figures, 4 tables, accepted by RAA_

</div>
<div id="authors">

X. Liu, et al. -- incl., <mark>S. Li</mark>, <mark>J. He</mark>

</div>
<div id="abstract">

**Abstract:** This paper presents an overview of the QUARKS survey,which stands for `Querying Underlying mechanisms of massive star formation with ALMA-Resolved gasKinematics and Structures'. The QUARKS surveyis observing 139 massive  clumps covered by 156 pointings at ALMA Band 6  ( $\lambda\sim$ 1.3 mm).In conjunction with data obtained from the ALMA-ATOMS survey at Band 3 ( $\lambda\sim$ 3 mm), QUARKS aims to  carry out an unbiased statisticalinvestigation of massive star formation process within  protoclusters down to a scale of 1000 au.This overview paper describes the observations and data reduction of the QUARKS survey,and gives a first look at an exemplar source, the mini-starburst Sgr B2(M).The wide-bandwidth (7.5 GHz) and high-angular-resolution ( $\sim 0.3\arcsec$ ) observations of the QUARKS surveyallow to resolve much more compact cores than could be done by the ATOMS survey, and to detectpreviously unrevealed fainter filamentary structures.The spectral windows cover transitions of species including CO, SO, $N_2$ D $^+$ , SiO, H $_{30}\alpha$ , $H_2$ CO, $CH_3$ CN and many othercomplex organic molecules, tracing gas components with different temperatures and spatial extents.QUARKS aims to deepen our understanding of several scientific topics of massive star formation, such as the mass transport within protoclusters by(hub-)filamentary structures, the existence of massive starless cores, the physical and chemical propertiesof dense cores within protoclusters, and the feedback from already formed high-mass young protostars. $\keywords{stars: formation --- stars: kinematics and dynamics --- ISM: clouds --- ISM: molecules}$

</div>

<div id="div_fig1">

<img src="tmp_2311.08651/./sgrb2_cores_labeled.png" alt="Fig6" width="100%"/>

**Figure 6. -** Zoom in image of the right panel of Figure \ref{fig_sgrb2_compareselfcal}.
The purple ellipses and black numbers marked the cores and substructures of Sgr B2(M) detected in the Band-6 continuum image
(Section \ref{sec_srgb2cores}).
The red rectangle marks the north arc of the Sgr B2(M).
The orange  ellipse in the lower-left corner represents the synthesized
beam size.  (*fig_sgrb2cores*)

</div>
<div id="div_fig2">

<img src="tmp_2311.08651/./sgrb2_band6_conti_selfcalcompare.png" alt="Fig4.1" width="50%"/><img src="tmp_2311.08651/./I08448_band6_conti_mergecompare.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** The ALMA Band 6 continuum of Sgr B2(M) (upper) and IRAS 08448-4343 (lower).
For Sgr B2(M), the upper right and upper left panels are the images
with and without selfcalibration, respectively.
For IRAS 08448-4343,  the lower left panel shows the  image stitched from two independently cleaned images,
and the lower right panel shows the  the image cleaned from the combined visibility data.
The two lower panels share the same colorbar.
In each panel, primary beam correction has been applied, and
the black small ellipse in the lower-left corner represents the synthesized
beam.  (*fig_sgrb2_compareselfcal*)

</div>
<div id="div_fig3">

<img src="tmp_2311.08651/./SgrB2_colorimages.png" alt="Fig5" width="100%"/>

**Figure 5. -** (a) Image of the ALMA Band 3  continuum emission of Sgr B2(M) with a resolution of $\sim 0.5$\arcsec$$ by [Ginsburg, Bally and Barnes (2018)]().
The beam size  is shown as an ellipse at the lower-left corner. The white contour shows the
ALMA Band 6 (at 1.3 mm) continuum emission from the QUARKS survey at the level of  5$\sigma$(2.5 mJy beam$^{-1}$).
(b) Image of the ALMA Band 6 continuum emission  from SM17  ([Sánchez-Monge, Schilke and Schmiedeke 2017]()) .
The fields of view of SM17 and the QUARKS survey are not identical,
but all 27 cores identified by [Sánchez-Monge, Schilke and Schmiedeke (2017)]() marked by the black crosses
are shown in this panel.
Contour is the same as in panel (a).
Note that the color map (_ rainbow_  with a power-law normalization with a power index of 0.4)
is similar to that of
Figure \ref{fig_sgrb2_compareselfcal}, where the  ALMA Band 6 continuum  of the QUARKS survey is shown.
(c) Three-color image composed by the Spitzer 8 (red), 4.5 (green), 3.4 (blue) $\mu$m continuum.
The white, cyan and black contours show the ALMA Band 6 continuum of the QUARKS survey, at the levels
2.5, 15, and 50 mJy beam$^{-1}$, respectively. (d) Three-color image composed of the
ALMA Band 6 continuum of the QUARKS survey in red, the ALMA Band 3 continuum  by [Ginsburg, Bally and Barnes (2018)]() in green,
and the VLA Band C (at $\sim$6 GHz) continuum by [Meng, Sánchez-Monge and Schilke (2022)]() in blue.
The cyan arrows pinpoint Cores 55 and 96 (see Figure \ref{fig_sgrb2cores} and Section \ref{sec_selfcali_check}).
The beam size of the QUARKS survey is shown as an ellipse at the lower-left corner.
 (*fig_colorimages*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2311.08651"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

321  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
